In [62]:
from operators import get_operator_aliases

def get_operator_param_names():
  return {
  'random_search': ['scale', 'distribution'],
  'central_force_dynamic': ['gravity', 'alpha', 'beta', 'dt'],
  'differential_mutation': ['expression', 'num_rands', 'factor'],
  'firefly_dynamic': ['distribution', 'alpha', 'beta', 'gamma'],
  'genetic_crossover': ['pairing', 'crossover', 'mating_pool_factor'],
  'genetic_mutation': ['scale', 'elite_rate', 'mutation_rate', 'distribution'],
  'gravitational_search': ['gravity', 'alpha'],
  'random_flight': ['scale', 'distribution'],
  'local_random_walk': ['probability', 'scale', 'distribution'],
  'random_sample': [],
  'spiral_dynamic': ['radius', 'angle', 'sigma'],
  'swarm_dynamic': ['factor', 'self_conf', 'swarm_conf', 'version', 'distribution']
  }

def get_inverse_operator_aliases():
  return {
  'RS': 'random_search',
  'CF': 'central_force_dynamic',
  'DM': 'differential_mutation',
  'FD': 'firefly_dynamic',
  'GC': 'genetic_crossover',
  'GM': 'genetic_mutation',
  'GS': 'gravitational_search',
  'RF': 'random_flight',
  'RW': 'local_random_walk',
  'RX': 'random_sample',
  'SD': 'spiral_dynamic',
  'PS': 'swarm_dynamic'
  }, {
    'g': 'greedy', 
    'd': 'all', 
    'm': 'metropolis', 
    'p': 'probabilistic'
  }


def get_inverse_str_args():
  return {
  'u': 'uniform',
  'rand': 'rand',
  'best': 'best',
  'current': 'current',
  'currenttobest': 'current-to-best',
  'randtobest': 'rand-to-best',
  'randtobestandcurrent': 'rand-to-best-and-current',
  'g': 'gaussian',
  'levy': 'levy',
  'rank': 'rank',
  's': 'single',
  'cost': 'cost',
  'r': 'random',
  't': 'tournament_2_100',
  'two': 'two',
  'b': 'blend',
  'l': 'linear_0.5_0.5',
  'i': 'inertial',
  'c': 'constriction'
  }
  
perturbator_alias, selector_alias = get_operator_aliases()
inverse_perturbator_alias, inverse_selector_alias = get_inverse_operator_aliases()
args_perturbators = get_operator_param_names()
inverse_args = get_inverse_str_args()



In [63]:
# Read operators and find their alias
collections = ['default.txt', 'basicmetaheuristics.txt']

encoded_heuristic_space = dict()
operators_string = dict()
for collection_file in collections:
    with open('./collections/' + collection_file, 'r') as operators_file:
        operators_string[collection_file] = [line.rstrip('\n') for line in operators_file]
        encoded_heuristic_space[collection_file] = [eval(line) for line in operators_string[collection_file]]

In [66]:
def convert_param(param_value):
  if type(param_value) == str:
    param_components = param_value.split('-')
    component_compressed = []
    for component in param_components:
      if len(component) > 4 and component != 'current':
        component_compressed.append(component[0])
      else:
        component_compressed.append(component)
    str_components = ''.join(component_compressed)
    return str_components, param_value
  else:
    return str(param_value), param_value
  

def get_inverse_arg(arg_value):
  try:
    # Return float number if possible
    return float(arg_value)
  except:
    # Decompress string
    return inverse_args[arg_value]

  
def compress_operator(operator):
  perturbator, args, selector = operator
  params = ';'.join([convert_param(arg)[0] for arg in args.values()])
  conv_perturbator = perturbator_alias[perturbator]
  conv_selector = selector_alias[selector]
  compress_operator = ','.join([conv_perturbator, params, conv_selector])
  return compress_operator


def decompress_operator(compress_operator):
  conv_perturbator, params, conv_selector = compress_operator.split(',')
  perturbator = inverse_perturbator_alias[conv_perturbator]
  selector = inverse_selector_alias[conv_selector]
  compressed_arg_values = params.split(';')
  arg_values = [get_inverse_arg(arg_value) for arg_value in compressed_arg_values]
  args = {}
  arg_names = args_perturbators[perturbator]
  for key, value in zip(arg_names, arg_values):
    args[key] = value
  return perturbator, args, selector
  

In [67]:
operator = ('differential_mutation',
 {'expression': 'random', 'num_rands': 1, 'factor': 1.0},
 'metropolis')

compressed_operator = compress_operator(operator)
decompressed_operator = decompress_operator(compressed_operator)
print(operator)

('differential_mutation', {'expression': 'random', 'num_rands': 1, 'factor': 1.0}, 'metropolis')


In [68]:
compress_operator(('differential_mutation',
 {'expression': 'random', 'num_rands': 1, 'factor': 1.0},
 'metropolis'))

'DM,r;1;1.0,m'

In [69]:
decompress_operator('RS,1.0;u,g')
decompress_operator('DM,r;1;1.0,m')

('differential_mutation',
 {'expression': 'random', 'num_rands': 1.0, 'factor': 1.0},
 'metropolis')

In [61]:

with open('collections/default_comprissed2.txt', 'w') as file:
  for operator in encoded_heuristic_space['default.txt']:
    compressed_op = compress_operator(operator)
    file.write(compressed_op + '\n')


In [70]:
limit_seq = 100

seqs, costs = [], []
for counting in range(1, 11):
  with open(f'vocabulary/seq_read_{counting}.txt', 'r', encoding='utf-8') as file:
    seqs = seqs + file.read().split('\n')  
  with open(f'vocabulary/score_{counting}.txt', 'r', encoding='utf-8') as file:
    costs = costs + file.read().split('\n')
    